In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf

import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json
import numpy as np


with open("info/design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

private file size: 2466


In [8]:
check_score(10)

10

In [9]:
ify_user = hf.shopify_printify(shop_config,'cbb')

In [14]:
cfbd_loader = hf.cfbp_handler(design_config["cfbd_api"])

In [15]:
teams = cfbd_loader.get_team_info()

In [29]:
teams.loc[teams.abrev == 'WSU']

,name,abrev,alt_color,color,division,id,logos,long_name,mascot,team_id
1712,Washington State,WSU,#6a747c,#94022a,fbs,265.0,http://a.espncdn.com/i/teamlogos/ncaa/500/265.png,None,Cougars,NaN


In [23]:
teams.loc[teams.logos == 'http://a.espncdn.com/i/teamlogos/ncaa/500/2089.png']

,name,abrev,alt_color,color,division,id,logos,long_name,mascot,team_id
805,LIU Post,LIU,None,#000000,None,2089.0,http://a.espncdn.com/i/teamlogos/ncaa/500/2089...,None,Pioneers,NaN


In [32]:
liu = requests.get('http://a.espncdn.com/i/teamlogos/ncaa/500/2089.png')
wsu = requests.get('http://a.espncdn.com/i/teamlogos/ncaa/500/265.png')

In [77]:
import json
import requests
from PIL import Image
from openai import OpenAI
import openai
import random
import math


def dalle_image(client, prompt, v, retry):
    fail = False
    try:
        response = client.images.generate(
            model=f"dall-e-{v}",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
    except:
        if retry:
            print('Retrying')
            try:
                response = client.images.generate(
                model=f"dall-e-{v}",
                prompt=prompt,
                size="1024x1024",
                quality="standard",
                n=1,
                )
            except:
                fail = True
        else:
            fail = True
    
    if fail:
        return 'filtered'


    image_url = response.data[0].url
    
    inter = requests.get(image_url, stream=True)

    image = Image.open(inter.raw)

    return image


def generate_main(prompt, dalle_key, test=True, dalle=3, retry=True):
    if test:
        print("using saved image because test mode")
        main_image = Image.open(r"test3.png")
    else:
        client = OpenAI(api_key=dalle_key)

        main_image = dalle_image(client, prompt, v=dalle, retry= retry)

        if main_image == 'filtered':
            print("image generation didnt work (likely content filter)")
            return None

        return main_image

In [78]:
generate_main('no no fuck bad inaproproate prompt', design_config['dalle_key'], test=False, dalle=2, retry=True)

Retrying


'filtered'

In [74]:
client = OpenAI(api_key=design_config['dalle_key'])

try:
    response = client.images.generate(
        model=f"dall-e-2",
        prompt='violence filtered, shit fuck',
        size="1024x1024",
        quality="standard",
        n=1,
    )
except Exception as e:
    

image_url = response.data[0].url

inter = requests.get(image_url, stream=True)

image = Image.open(inter.raw)


BadRequestError: Error code: 400 - {'error': {'code': 'content_policy_violation', 'message': 'Your request was rejected as a result of our safety system. Your prompt may contain text that is not allowed by our safety system.', 'param': None, 'type': 'invalid_request_error'}}

In [3]:
# check if printify ptoduct is posted
headers_printify = {
    "Authorization": f"Bearer {shop_config['printify_access']}",
    "Content-Type": "application/json",
}
headers_shopify = {
            "X-Shopify-Access-Token": shop_config["shopify_access"],
            "Content-Type": "application/json",
        }

In [14]:
products_link = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/products.json?limit=250"
response = requests.get(products_link, headers=headers_shopify)
response.headers["X-Shopify-Shop-Api-Call-Limit"]

'1/40'

In [15]:
call_limit = response.headers["X-Shopify-Shop-Api-Call-Limit"]
call_split = [int(x) for x in call_limit.split('/')]
remaining_calls = call_split[1]-call_split[0]

In [10]:
print(len(prods))
print(len(prods2))

472
236


In [ ]:
import time

In [4]:
graphql_url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2023-04/graphql.json"


In [5]:
big_body ="""
            {
            product(id:"gid://shopify/Product/prod_id") {
                title
                media(first:5) {
                edges {
                    node {
                    ... fieldsForMediaTypes
                    }
                }
                }
            }
            }

            fragment fieldsForMediaTypes on Media {
            alt
            mediaContentType
            preview {
                image {
                id
                altText
                url
                }
            }
            status
            ... on Video {
                id
                sources {
                format
                height
                mimeType
                url
                width
                }
                originalSource {
                format
                height
                mimeType
                url
                width
                }
            }
            ... on ExternalVideo {
                id
                host
                embeddedUrl
            }
            ... on Model3d {
                sources {
                format
                mimeType
                url
                }
                originalSource {
                format
                mimeType
                url
                }
            }
            ... on MediaImage {
                id
                image {
                altText
                url
                }
            }
            }
            """

In [6]:
prod_id = 8368514007261

In [7]:
all_p = ify_user.recur_get_products()

In [9]:
for x in all_p:
    if x['id'] == prod_id:
        prod = x

In [12]:
prod_gql = prod["admin_graphql_api_id"]
body = big_body.replace("prod_id", str(prod_id))
# now that I have the images I can reorder them
url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
# from this response i can get my images
response = requests.post(
    url=url, headers=headers_shopify, json={"query": body}
)

images = json.loads(response.text)["data"]["product"]["media"]["edges"]
image0 = images[0]["node"]["id"]
image2 = images[2]["node"]["id"]

In [14]:
title = prod['title']

In [15]:
alt_text_query = "mutation productUpdateMedia($media: [UpdateMediaInput!]!, $productId: ID!) { productUpdateMedia(media: $media, productId: $productId) { media { alt } } }"

alt_text_var = {
"media": [
    {
    "alt": title+' back view',
    "id": image0
    }
],
"productId": prod_gql
}

alt_resp = requests.post(
    graphql_url, headers=headers_shopify, json={"query": alt_text_query, "variables": alt_text_var}
)

In [ ]:
import numpy as np

In [16]:
'currentlyAvailable' in str(alt_resp.headers)

False

In [26]:
json.loads(alt_resp.text)['extensions']['cost']['throttleStatus']['currentlyAvailable'] < 100:

False

In [32]:
alt_resp.headers["extensions"]

KeyError: 'extensions'